In [1]:
import os 


from dotenv import load_dotenv, find_dotenv
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

from langchain_pinecone import PineconeVectorStore
from langchain_openai import ChatOpenAI

from langchain_community.embeddings import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.runnables import RunnableParallel, RunnablePassthrough


In [8]:
vectorstore = PineconeVectorStore.from_existing_index(index_name='sql-sample-rag-test', embedding=OpenAIEmbeddings(),  text_key='masked_question')

In [9]:
_ = load_dotenv(find_dotenv())

In [10]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [11]:
# Re-rank
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [12]:
# RAG prompt
template = """Find the related sql question query examples based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI()
chain = (
    RunnableParallel(
        {"context": compression_retriever, "question": RunnablePassthrough()}
    )
    | prompt
    | model
    | StrOutputParser()
)



In [13]:
chain.invoke('return me the authors who have papers in VLDB conference before 2002 .')

"SELECT author\nFROM papers\nWHERE conference = 'VLDB'\nAND year < 2002;"